# Tutorial 4: Manipulation with RRT

In this tutorial, we are going to plan some configuration space (C-space) trajectories of the robot. And then, we are going to discuss the RRT method together with the IK method that we have covered in the previous tutorial.

# Rapidly-Exploring Random Trees (RRT)

## Import Packages and some code

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')
%matplotlib notebook

# code from the preivous tutorial
class TwoLinkArm:
    
    def __init__(self):
        # Set parameters for the 2-link planar arm
        
        self.theta1 = np.deg2rad(0)
        self.theta2 = np.deg2rad(90)
        self.l1 = 5
        self.l2 = 3

    def forwardKinematics(self, theta1, theta2):
        # Define the homogeneous transformation matrices for the 2-link planar arm
        self.theta1 = theta1
        self.theta2 = theta2

        self.t01 = np.matrix([[np.cos(self.theta1), -np.sin(self.theta1), 0],
                        [np.sin(self.theta1), np.cos(self.theta1), 0],
                        [0, 0, 1]])
        self.t12 = np.matrix([[np.cos(self.theta2), -np.sin(self.theta2), self.l1],
                              [np.sin(self.theta2),  np.cos(self.theta2), 0],
                              [0, 0, 1]])
        self.t3end = np.matrix([[1, 0, self.l2], 
                                [0, 1, 0], 
                                [0, 0, 1]])
        self.t0end = self.t01*self.t12*self.t3end
        
        # Find the transformation matrices for joint 2 and joint 3
        t02 = self.t01*self.t12
        
        # Find the x, y coordinates for joints 2. Put them in a list j2 = [x,y]
        j2 = [t02[0,2],t02[1,2]]
        
        endeff = [self.t0end[0,2],self.t0end[1,2]]
        return j2,endeff
    
# gif animation plotter
def plotArm2_2link(jnt2pos, endEffectPos, target=None, step=None, fig=None, camera=None):
    # set up figure
    ax = fig.add_subplot(111, autoscale_on=False,
                         xlim=(-10, 10), ylim=(-10, 10))
    ax.grid()
    line, = ax.plot([], [], 'o-', lw=4, mew=5, color='lightblue')

    if step is not None:
        step = np.array(step)
        plt.scatter(step[:, 0],step[:, 1], color='blue', marker='o')
        plt.plot(step[:, 0],step[:, 1], color='blue')
    if target is not None:
        target = np.array(target)
        plt.scatter(target[:, 0],target[:, 1], color='red', marker='o')
    
    line.set_data([], [])
    x = np.array([0, jnt2pos[0], endEffectPos[0]])
    y = np.array([0, jnt2pos[1], endEffectPos[1]])
    line.set_data((x,y))

    camera.snap()

## RRT Algorithm
Rapidly-Exploring Randoms Trees (RRT) is a sampling-based planning algorithm which makes use of probabilistic methods and a roadmap. The roadmap shown below is created by RRT and represents a network of potential obstacle free paths through the environment. The main advantage of RRT in comparison to algorithms such as A* is the use of random sampling which greatly reduces computational cost when analysing the map. A limitation of this approach is that computed paths are non-optimal, due to the random sampling of nodes.

The graph or roadmap below is an abstract representation of a set of objects or nodes/vertices connected by links or edges. The edges can be directed, in this case the starting point is $q_1$. A sequence of connected edges is a path. 

## RRT Pseudocode

<img src="images/rrt_qnear.png" style="width: 250px;">

<img src="images/rrt_pseudo.png" style="width: 700px;">

Inputs: 
* Initial configuration/position $q_{init}$
* Number of nodes, K. In our case we use a while loop until one of the nodes is within a certain radius of the goal.
* Incremental distance or step size $\Delta q$

Looping until target is reached:
1. A random configuration $q_{rand}$ is chosen. $q_{rand}$ is discarded if it lies within an obstacle.
2. The closest node/vertex $q_{near}$ is found. Near being defined in terms of a cost function that includes distance and orientation. $q_{near}$ will not be added to the graph if the path from $q_{near}$ to $q_{rand}$ intersects an obstacle. 
3. The point $q_{new}$ is added, which is the distance the robot can move in a fixed path towards $q_{rand}$ from $q_{near}$.

## Voronoi Bias
In the Voronoi diagram below we can see each cell or region corresponds to a site within the cell and consists of all points that are closer to its site than to any other site. The edges of the cells are the points that are equidistant to the two nearest sites. 

A key property of the RRT algorithm is that it uses what we call a "Voronoi bias" in the exploration process by expanding the node in the tree that is closest ($q_{near}$) to the random point ($q_{rand}$) at each iteration. By using random samples, the probability that a vertex/node is chosen is proportional to the volume of its Voronoi region. This encourages tree growth to the larger unexplored regions. 

<img src="images/voronoi.png" style="width: 350px;">

<img src="images/rrt_tree.gif" style="width: 400px;">

### Define RRT Class (no coding needed)

In [2]:
#Global starting configuration
start = (np.deg2rad(45), np.deg2rad(45))
#Global target configuration
target = (np.deg2rad(-135), np.deg2rad(10))

In [3]:
#Global configuration
RADIUS_OBSTACLE = np.deg2rad(5)
RADIUS_TARGET = np.deg2rad(25)
RRT_EXTEND_DIST = np.deg2rad(25)
SMOOTHING_ITERATIONS = 200
SMOOTHING_STEP = 0.1

#Global robot joint limits configuration
JOINT1_LIM = np.array([-np.pi+np.deg2rad(30), np.pi-np.deg2rad(30)])
JOINT2_LIM = np.array([-np.pi+np.deg2rad(30), np.pi-np.deg2rad(30)])

class RRT():
    def __init__(self):
        # circle obstacles list [(theta_1, theta_2, theta_3)]
        self.obstacles = []
        # tree nodes list [((theta_1, theta_2, theta_3), parent_index)]
        self.nodes = [(start, None)]
        
    #Add a line of obstacles between the two given points
    def generate_obstacles_line(self, q1, q2):
        q1, q2 = np.array(q1), np.array(q2)
        length = np.linalg.norm(q2-q1)
        vect = (q2 - q1)/length
        for l in np.arange(0.0, length, 1.9*RADIUS_OBSTACLE):
            self.obstacles += [(q1[0]+l*vect[0], q1[1]+l*vect[1])]

    #Return True if the given point is colliding 
    def check_point_collision(self, q):
        for o in self.obstacles:
            dist = np.sqrt(np.power(q[0]-o[0], 2) + np.power(q[1]-o[1], 2))
            if dist <= RADIUS_OBSTACLE:
                return True
        return False

    #Return True if the segment between the two 
    #given points is colliding the obstacles
    def check_segment_collision(self, q1, q2):
        q1, q2 = np.array(q1), np.array(q2)
        length = np.linalg.norm(q2-q1)
        vect = (q2 - q1)/length
        for l in np.arange(0.0, length, 0.6*RADIUS_OBSTACLE):
            q = (q1[0]+l*vect[0], q1[1]+l*vect[1])
            if self.check_point_collision(q):
                return True
        return False

    #Return a random point in the working space
    def sample_point(self):
        theta1 = random.uniform(-np.pi, np.pi)
        theta2 = random.uniform(-np.pi, np.pi)
        return (theta1,theta2)

    #Return the index of the closest node in RRT from given point
    def find_nearest_neighbour(self, q):
        index = 0
        dist_min = float('inf')
        for (i, n) in enumerate(self.nodes):
            dist = np.sqrt(np.power(q[0]-n[0][0], 2) + np.power(q[1]-n[0][1], 2))
            if dist < dist_min:
                dist_min = dist;
                index = i
        return index
    
    def find_new_reachable_pt(self, q_sample, q_near):
        
        dist = np.sqrt(np.power(q_sample[0]- q_near[0], 2) + np.power(q_sample[1]- q_near[1], 2))
        if dist > RRT_EXTEND_DIST:
            q_normalized_x = q_near[0] + RRT_EXTEND_DIST*(q_sample[0]- q_near[0])/dist
            q_normalized_y = q_near[1] + RRT_EXTEND_DIST*(q_sample[1]- q_near[1])/dist
        else:
            q_normalized_x = q_sample[0]
            q_normalized_y = q_sample[1]
        q_new = (q_normalized_x, q_normalized_y)

        return q_new
    

### Implementation of the RRT Algorithm

Assume we have a faulty 2-arm manipulator that cannot reach some configurations. Below, we are creating such C-space with some unreachable areas. 

The unreachable area will be illustrated using blue circles.

In [4]:
rrt_plan = RRT()

# Define non-reachable configuration areas
rrt_plan.generate_obstacles_line((JOINT1_LIM[0], JOINT2_LIM[0]), (JOINT1_LIM[0], JOINT2_LIM[1]))
rrt_plan.generate_obstacles_line((JOINT1_LIM[1], JOINT2_LIM[0]), (JOINT1_LIM[1], JOINT2_LIM[1]))
rrt_plan.generate_obstacles_line((JOINT1_LIM[0], JOINT2_LIM[0]), (JOINT1_LIM[1], JOINT2_LIM[0]))
rrt_plan.generate_obstacles_line((JOINT1_LIM[0], JOINT2_LIM[1]), (JOINT1_LIM[1], JOINT2_LIM[1]))
rrt_plan.generate_obstacles_line((-0.8, -0.1), (-1.1, 1.8))
rrt_plan.generate_obstacles_line((1, -1.), (-1.1, 1.8))
rrt_plan.generate_obstacles_line((-0.8, -0.1), (1, -1.))

#Sample and grow the RRT until target is reached
is_reached = False
while not is_reached:
    
    # Sample a new point  
    q_sample = rrt_plan.sample_point()
    
    # Check that the new point is collision free
    if not rrt_plan.check_point_collision(q_sample):

        # Find the index of the nearest node (q_near) in the graph    
        index = rrt_plan.find_nearest_neighbour(q_sample)
        q_near = rrt_plan.nodes[index][0]

        # Find the closest feasible point to the randomly sampled point (q_new)
        q_new = rrt_plan.find_new_reachable_pt(q_sample, q_near)
        
        # Check if the edge is collision free
        if not rrt_plan.check_segment_collision(q_new, q_near):
        
            rrt_plan.nodes += [(q_new, index)]

            # Check if the goal has been reached
            if np.sqrt(np.power(q_new[0]-target[0], 2) + np.power(q_new[1]-target[1], 2)) < RADIUS_TARGET:
                is_reached = True

#### Extracting the planned path

<img src="images/PATH_diag.png" style="width:800px">

In [7]:
# Retrieve computed path from the tree
# (get the path from the target, by traversing the list in the reversed order.)
path = []
### START YOUR CODE HERE

# remember to append the starting point

### END YOU CODE HERE

# Reversing the path to be correct.
path.reverse()

In [8]:
#Path smoothing
for k in range(SMOOTHING_ITERATIONS):
    index1 = random.randint(0, len(path)-1)
    index2 = random.randint(0, len(path)-1)
    if index1 != index2 \
        and not rrt_plan.check_segment_collision(path[index1], path[index2]) \
        and np.linalg.norm(np.array(path[index1]) - np.array(path[index2])) > 1.0:
        if index1 < index2:
            index_low = index1
            index_up = index2
        else:
            index_low = index2
            index_up = index1
        middle = []
        deltax = (path[index_up][0]-path[index_low][0])
        deltay = (path[index_up][1]-path[index_low][1])
        for l in np.arange(SMOOTHING_STEP, 1.0-SMOOTHING_STEP, SMOOTHING_STEP):
            middle += [(path[index_low][0]+l*deltax, path[index_low][1]+l*deltay)]
        path = path[:index_low+1] + middle + path[index_up:]

Now, we have got the planned trajectory stored in the variable ``path``

In [9]:
print(f"the trajectory length is {len(path)}")

the trajectory length is 28


### Plotting the RRT Roadmap

In [10]:
%matplotlib inline

node_prog = np.linspace(7, len(rrt_plan.nodes), 10, int)

fig = plt.figure(figsize=(4,4))
plt.axis('scaled')
plt.grid()
plt.gcf().gca().set_xlim(tuple(JOINT1_LIM))
plt.gcf().gca().set_ylim(tuple(JOINT2_LIM))
camera = Camera(fig)

# Plot growing tree
for j in range(len(node_prog)): 
    
    #Draw obstacles
    for o in rrt_plan.obstacles:
        circle = plt.Circle(o, RADIUS_OBSTACLE, color='b', fill=False)
        plt.gcf().gca().add_artist(circle)

    #Draw start and target points
    circle_start_1 = plt.Circle(start, RADIUS_TARGET, color='g', alpha=0.5)
    circle_start_2 = plt.Circle(start, RADIUS_OBSTACLE, color='g')
    circle_target_1 = plt.Circle(target, RADIUS_TARGET, color='r', alpha=0.5)
    circle_target_2 = plt.Circle(target, RADIUS_OBSTACLE, color='r')
    plt.gcf().gca().add_artist(circle_start_1)
    plt.gcf().gca().add_artist(circle_start_2)
    plt.gcf().gca().add_artist(circle_target_1)
    plt.gcf().gca().add_artist(circle_target_2)
    
    #Draw tree
    for n in rrt_plan.nodes[0:int(node_prog[j])]:
        if (n[1] != None):
            x1 = n[0][0]
            y1 = n[0][1]
            parent = n[1]
            x2 = rrt_plan.nodes[parent][0][0]
            y2 = rrt_plan.nodes[parent][0][1]
            plt.plot([x1, x2], [y1, y2], color='y', marker='.')
    
    # Final plot
    if(j >= len(node_prog)-1):
    
        #Draw Path
        for i in range(len(path)):
            if i > 0:
                plt.plot([path[i-1][0], path[i][0]], [path[i-1][1], path[i][1]], color='r', marker='.')
    camera.snap()
    
plt.close()
animation = camera.animate()
animation.save('animated_RRT.gif', writer = 'imagemagick')

MovieWriter imagemagick unavailable; using Pillow instead.


In [12]:
# show the animation

#random code to make sure the gif is reloaded (jupyter bug)
import random
__counter__ = random.randint(0,2e9)


from IPython.display import HTML
display(HTML('<img src="animated_RRT.gif?%d">'% __counter__))

### Manipulating the arm
Now, let's manipulate our toy manipulator according to the planned RRT path. 

In [13]:
arm = TwoLinkArm()

# Set up the animation generator
fig = plt.figure(figsize=(4,4))
camera = Camera(fig)

EFTrace = []
for (theta1, theta2) in path:
    # Manipulate the arm using the 'arm' object
    joint2pos, endEffectorPos = arm.forwardKinematics(theta1, theta2)
    EFTrace += [endEffectorPos]
    # Plot the pose of the arm
    plotArm2_2link(joint2pos, endEffectorPos, target=None, step=EFTrace, fig=fig, camera=camera)
    
plt.close()
animation = camera.animate()
animation.save('animated_RRT_arm.gif', writer = 'imagemagick')

# show the animation
__counter__ = random.randint(0,2e9)
display(HTML('<img src="animated_RRT_arm.gif?%d">'% __counter__))

MovieWriter imagemagick unavailable; using Pillow instead.


Well done! There you have it, trajectory planning with RRT. 

Now, think of the following questions:

1. What is the difference between "the task space trajectory" and "the C-space trajectory"?
2. Continuing from (1), when are they prefered over the other one?

To provide you with a reference, here is the IK manipulator from the early tutorial:

<img src="images/animated_IK.gif" style="width: 300px;">

Here is another example in 3-D: task space trajectory (left), C-space trajectory (right)

<img src="images/trap_vs_poly.gif" style="width: 450px;">

For extra information, e.g. the velocity profile, see in: https://blogs.mathworks.com/student-lounge/2019/11/06/robot-manipulator-trajectory/

(You will find hits of the thinking questions above in this link)

You are also welcomed to use this code to try with different initial/target positions. The goal of this tutorial is to give you a more concrete intuition of the RRT algorithm. If you are still, keen, try to write a generic RRT algorithm for planning in N-D C-spaces, based on the code provided here :)

**References**

Corke, P., 2017. Robotics, vision and control: fundamental algorithms in MATLAB® second, completely revised (Vol. 118). Springer.
https://en.wikipedia.org/wiki/Rapidly-exploring_random_tree
